In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import matplotlib.pyplot as plt
import colorcet as cc
import networkx as nx
import pandas as pd
import plotly.graph_objects as go
import sys

sys.path.append("/Users/jlq293/Projects/Study-1-Bibliometrics/src/main_path/")

from MainPathReader import MainPathReader
from MainPathDataAssigner import MainPathDataAssigner

AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

In [ ]:
p = "../data/06-clustered-df/alpha0.3_k20_res0.006.pkl"  # cluster_alpha0.3_k20_res0.006
df = pd.read_pickle(p)

p_summary = "../output/tables/cluster-explorer/FinalSelect/SingleSolExplorer_alpha0.3_k20_res0.006.xlsx"
# read sheet 1
df_summary = pd.read_excel(p_summary, sheet_name=0)

Cannot be read by networkx, so reading in from scratch

# read in main path data from pajek


In [ ]:
path_to_main_path = (
    "../data/08-main-paths/10. Key-Route Global Main Path [200-250] of N5 (185).net"
)
mp_reader = MainPathReader(
    path_to_main_path, node_attributes=["eid", "unique_auth_year"]
)
Gmp = mp_reader.get_graph()
list(Gmp.nodes(data=True))[:2]

FileNotFoundError: [Errno 2] No such file or directory: '../data/08-main-paths/10. Key-Route Global Main Path [200-250] of N5 (185).net'

# Add Info from df to mp


In [ ]:
attr_to_assign = [
    "title",
    "citedby_count",
    "unique_auth_year",
    "doi",
    "year",
    "cluster_alpha0.3_k20_res0.006",
    "centrality_alpha0.3_k20_res0.006",
]


data_assigner = MainPathDataAssigner(Gmp, df, attr_to_assign)
Gmp = data_assigner.process_mp()
print(list(Gmp.nodes(data=True))[:2])

[('1', {'label': '"family_2_62"', 'eid': '2-s2.0-0020047901;2-s2.0-0019989539', 'unique_auth_year': 'Boeck_1982;Pedersen_1982', 'title': 'Studies on Acute Toxicity and Drug Levels of Citalopram in the Dog;Citalopram, a selective serotonin reuptake inhibitor: Clinical antidepressive and long-term effect - a phase II study', 'citedby_count': '34;61', 'doi': '10.1111/j.1600-0773.1982.tb00959.x;10.1007/BF00464566', 'year': '1982;1982', 'cluster_alpha0.3_k20_res0.006': '47;76', 'centrality_alpha0.3_k20_res0.006': '0.073681300627201;0.3242186570081556'}), ('2', {'label': '"47"', 'eid': '2-s2.0-0020069956', 'unique_auth_year': 'Baker_1982', 'title': 'The effects of the specific uptake inhibitor Lu 10-171 (Citalopram) upon brain indoleamine stores in the maturing mouse', 'citedby_count': '11', 'doi': '10.1016/0306-3623(82)90013-1', 'year': '1982', 'cluster_alpha0.3_k20_res0.006': '67', 'centrality_alpha0.3_k20_res0.006': '0.06512483207962619'})]


# plot the main path


In [ ]:
from MainPathPlotterStatic import MainPathPlotterStatic
from MainPathPlotterInteractive import MainPathPlotterInteractive

In [ ]:
import colorcet as cc
import matplotlib.pyplot as plt
import networkx as nx
import plotly.graph_objects as go
import plotly.io as pio
import numpy as np


class MainPathPlotterInteractive:
    def __init__(self, G, cluster_col, label_col):
        self.G = G
        self.cluster_col = cluster_col
        self.label_col = label_col

    @staticmethod
    def add_line_breaks(text, char_limit=100):
        words = text.split()
        line = ""
        lines = []
        for word in words:
            if len(line + " " + word) <= char_limit:
                line += " " + word
            else:
                lines.append(line.strip())
                line = word
        if line:
            lines.append(line.strip())
        return "<br>".join(lines)

    def adjust_overlap(self, min_dist=0.1, max_iterations=100):
        """
        Adjusts the positions to reduce overlap.

        :param pos: Dictionary of positions keyed by node.
        :param min_dist: Minimum desired distance between nodes.
        :param max_iterations: Maximum number of iterations to perform adjustments.
        :return: Adjusted positions.
        """
        for _ in range(max_iterations):
            moved = False
            for node1 in self.pos:
                for node2 in self.pos:
                    if node1 != node2:
                        # Calculate the distance between two nodes
                        delta = np.array(self.pos[node1]) - np.array(self.pos[node2])
                        dist = np.sqrt(np.sum(delta**2))
                        # If the nodes are too close, push them apart
                        if dist < min_dist:
                            moved = True
                            displacement = delta / dist * (min_dist - dist) / 2
                            self.pos[node1] += displacement
                            self.pos[node2] -= displacement
            # If no nodes were moved in this iteration, stop the adjustment
            if not moved:
                print(f"Adjustment finished after {_} iterations")
                break
        return self.pos

    def clean_years(self):
        """
        Convert year attribute to integer, if not already
        if multiple years are present, the mean is taken
        """
        # Convert year attribute to integer, if not already
        for node, data in self.G.nodes(data=True):
            if isinstance(data["year"], str):
                mean_year = sum(map(int, data["year"].split(";"))) / len(
                    data["year"].split(";")
                )
                data["year"] = int(mean_year)

        # Sort nodes based on year attribute
        nodes_sorted_by_year = sorted(
            self.G.nodes(data=True), key=lambda x: x[1]["year"], reverse=True
        )

        # Get the range of years
        min_year = min([data["year"] for node, data in nodes_sorted_by_year])
        max_year = max([data["year"] for node, data in nodes_sorted_by_year])

        return min_year, max_year, nodes_sorted_by_year

    def hover_texts(self):
        hover_texts = []
        for node, data in self.G.nodes(data=True):
            wrapped_title = self.add_line_breaks(data["title"])
            hover_title_cluster = (
                f"Cluster: {data[self.cluster_col]}<br>{wrapped_title}"
            )
            hover_title_cluster_cited = f"Cluster: {data[self.cluster_col]}<br>Cited by: {data['citedby_count']}<br>{wrapped_title}"
            hover_texts.append(hover_title_cluster_cited)
        return hover_texts

    def cluster_color_dict(self):
        """
        Creates a dictionary mapping clusters to distinct colors using the colorcet library,
        and returns a list of colors for each node based on its cluster.

        Returns:
            list: A list of color strings in hex format for each node in the graph, based on its cluster.
        """
        clusters = nx.get_node_attributes(self.G, self.cluster_col).values()
        unique_clusters = sorted(set(clusters))

        # Using colorcet to get a colormap with enough distinct colors
        # Adjust the colormap if needed to suit your preferences for color variety and distinctness
        color_list = cc.glasbey_dark[: len(unique_clusters)]

        # Creating a color dictionary mapping each cluster to a color
        color_dict = {
            cluster: color_list[i] for i, cluster in enumerate(unique_clusters)
        }

        # Generating a list of colors for each node based on its cluster
        colors = [color_dict[cluster] for cluster in clusters]
        return colors

    def node_labels(self):
        node_labels = []
        for node, data in self.G.nodes(data=True):
            node_labels.append(data[self.label_col])
        return node_labels

    def edge_positions(self):
        edge_x = []
        edge_y = []
        for edge in self.G.edges():
            x0, y0 = self.pos[edge[0]]
            x1, y1 = self.pos[edge[1]]
            edge_x.append(x0)
            edge_x.append(x1)
            edge_x.append(None)
            edge_y.append(y0)
            edge_y.append(y1)
            edge_y.append(None)
        return edge_x, edge_y

    def get_edge_trace(self):
        edge_x, edge_y = self.edge_positions()
        edge_trace = go.Scatter(
            x=edge_x,
            y=edge_y,
            line=dict(width=0.7, color="#000000"),
            hoverinfo="none",
            mode="lines",
        )
        return edge_trace

    def get_node_trace(self):
        node_x, node_y = zip(*self.pos.values())
        hover_texts = self.hover_texts()
        colors = self.cluster_color_dict()
        node_labels = self.node_labels()
        node_trace = go.Scatter(
            x=node_x,
            y=node_y,
            mode="markers+text",
            hoverinfo="text",
            hovertext=hover_texts,
            text=node_labels,  # Permanent labels for nodes
            textposition="middle center",
            marker=dict(
                showscale=False,
                opacity=0.3,  # Decreased opacity of nodes for the desired transparency
                # colorscale='YlGnBu',
                size=20,
                colorbar=dict(
                    thickness=15,
                    title="Node Connections",
                    xanchor="left",
                    titleside="right",
                ),
                line_width=2,
                color=colors,
            ),
        )
        return node_trace

    def plot_network_on_timeline_interactive(
        self, savingpath=None, return_fig=True, adjust_overlap=True
    ):
        # Convert year attribute to integer, if not already
        # min_year, max_year, nodes_sorted_by_year = self.clean_years()

        self.pos = nx.kamada_kawai_layout(
            self.G,
            weight=None,
            scale=5.0,
            center=None,
            dim=2,
        )  # Use Kamada-Kawai layout for better node positioning

        if adjust_overlap:
            self.pos = self.adjust_overlap(0.15, 100)

        edge_trace = self.get_edge_trace()

        node_trace = self.get_node_trace()
        width = 1000
        height = 1500
        if len(self.G.nodes) > 190:
            width = 1300
            height = 1500
        # Figure Definition
        fig = go.Figure(
            data=[edge_trace, node_trace],
            layout=go.Layout(
                width=width,  # Example width
                height=height,  # Example height
                showlegend=False,  # Adjust based on your need to show or hide the legend
                hovermode="closest",
                hoverlabel=dict(  # Customizing the hover label's appearance
                    bgcolor="rgba(249, 98, 186, 0.11)",  # Setting the background color to a semi-transparent gray
                    bordercolor="rgba(0, 0, 0, 0.14)",
                    font=dict(color="black"),  # Text color in the hover label
                ),
                margin=dict(b=0, l=0, r=0, t=40),
                xaxis=dict(
                    showline=False,  # Hides the axis line
                    showgrid=False,
                    zeroline=False,
                    showticklabels=False,  # Hide tick labels
                    visible=False,  # Makes the whole axis (including the space for it) invisible
                ),
                yaxis=dict(
                    showline=False,  # Hides the axis line
                    showgrid=False,
                    zeroline=False,
                    showticklabels=False,  # Hide tick labels
                    visible=False,  # Makes the whole axis (including the space for it) invisible
                ),
                dragmode="zoom",
            ),
        )

        plt.tight_layout()
        if savingpath:
            if savingpath.endswith(".html"):  # Use 'savingpath' consistently
                fig.write_html(savingpath)
            elif savingpath.endswith(".png"):
                fig.write_image(savingpath, dpi=300)
        fig.show()
        if return_fig:
            return pio.to_html(fig, full_html=False, include_plotlyjs="cdn")


# example usage
# plotter = MainPathPlotterInteractive(graph, "cluster")
# plotter.plot_network_on_timeline_interactive(return_fig=False, adjust_overlap=True)
# plotter.plot_network_on_timeline_interactive(savingpath="network_timeline.html")
# plotter.plot_network_on_timeline_interactive(savingpath="network_timeline.png")

In [ ]:
plotter = MainPathPlotterInteractive(
    Gmp, "cluster_alpha0.3_k20_res0.006", "unique_auth_year"
)
fig = plotter.plot_network_on_timeline_interactive(return_fig=True, adjust_overlap=True)

<Figure size 640x480 with 0 Axes>

In [ ]:
clusters = len(
    set(nx.get_node_attributes(Gmp, "cluster_alpha0.3_k20_res0.006").values())
)

In [ ]:
clusters

36

In [ ]:
# mean year of cluster 5
df[df["cluster_alpha0.3_k20_res0.006"] == 4]["year"].mean()

2015.9114114114113

In [ ]:
# dataframe from Gmp
nodesmp = pd.DataFrame(**Gmp.nodes(data=True))

TypeError: pandas.core.frame.DataFrame() argument after ** must be a mapping, not NodeDataView

In [ ]:
# Convert the nodes and their attributes into a pandas DataFrame
nodesmp = pd.DataFrame.from_dict(dict(Gmp.nodes(data=True)), orient="index")

# If you want to include the node ID as a column as well, you can reset the index:
nodesmp.reset_index(inplace=True)
nodesmp.rename(columns={"index": "node_id"}, inplace=True)

nodesmp  # ["cluster_alpha0.3_k20_res0.006"].value_counts()

,node_id,label,x,y,size,shape,eid,unique_auth_year
0,1,"""family_2_62""",0.6974,0.9217,0.5000,ellipse,2-s2.0-0020047901;2-s2.0-0019989539,Boeck_1982;Pedersen_1982
1,2,"""47""",0.6737,0.9343,0.5000,ellipse,2-s2.0-0020069956,Baker_1982
2,3,"""family_33_36_37_40""",0.6782,0.9152,0.5000,ellipse,2-s2.0-0020425640;2-s2.0-0020416931;2-s2.0-002...,Overo_1982_2;Hyttel_1982;Ofsti_1982;Overo_1982
3,4,"""43""",0.6676,0.8984,0.5000,ellipse,2-s2.0-0020363953,Gastpar_1982
4,5,"""16""",0.2052,0.8995,0.5000,ellipse,2-s2.0-0019996341,Magnussen_1982
...,...,...,...,...,...,...,...,...
180,181,"""35944""",0.3830,0.0639,0.5000,ellipse,2-s2.0-85149225801,Venkatachalam_2023
181,182,"""36607""",0.4687,0.1007,0.5000,ellipse,2-s2.0-85161263006,Ferreira_2023
182,183,"""36624""",0.3755,0.0457,0.5000,ellipse,2-s2.0-85174155976,Fong_2023
183,184,"""36764""",0.4871,0.0906,0.5000,ellipse,2-s2.0-85177791552,Polverino_2023


In [ ]:
19. Key-Route Global Main Path [200-300] of N5 (237).net
22. Key-Route Local Main Path [100-200] [0.010] of N5 (213).net

10. Key-Route Global Main Path [200-250] of N5 (185).net

SyntaxError: invalid syntax (504417644.py, line 1)

In [ ]:

# Get the list of nodes from the graph
all_nodes = list(G.nodes())

# Set a random seed for reproducibility
random.seed(42)

# Sample 1000 nodes randomly
sample_nodes = random.sample(all_nodes, 1000)

# Create a subgraph containing only the sampled nodes
G_sample = G.subgraph(sample_nodes)

In [34]:
clusters = [
    #8,9,10,11,12,13,14,
    15,16,17,18,19,20]

# Create a subgraph containing only the nodes in the selected clusters
G_cluster = G.subgraph([node for node, data in G.nodes(data=True) if data["cluster"] in clusters])

print(f"Number of nodes in the selected clusters: {len(G_cluster.nodes)}")

# color dict in this format 0xff0000
color_dict = {
    0: "0xff0000",
    1: "0x0000ff",
    2: "0x00ff00",
    3: "0xffff00",
    4: "0x800080",
    5: "0xffa500",
    6: "0x00ffff",
    7: "0xff00ff",
    8: "0x00ff00",
    9: "0xffc0cb",
    10: "0x008080",
    11: "0xe6e6fa",
    12: "0xa52a2a",
    13: "0xf5f5dc",
    14: "0x800000",
    15: "0x98ff98",
    16: "0x808000",
    17: "0xfdd9b5",
    18: "0x000080",
    19: "0x808080",
    20: "0xff00ff",
}



Number of nodes in the selected clusters: 2863


In [37]:
import pandas as pd
import networkx as nx
G = nx.read_graphml("/Users/jlq293/Projects/Study-1-Bibliometrics/data/07-clustered-graphs/alpha0.3_k20_res0.005.graphml")

clusters = [
    #8,9,10,11,12,13,14,
    15,16,17,18,19,20]

# Create a subgraph containing only the nodes in the selected clusters
G_sample = G.subgraph([node for node, data in G.nodes(data=True) if data["cluster"] in clusters])

print(f"Number of nodes in the selected clusters: {len(G_sample.nodes)}")

# color dict in this format 0xff0000
color_dict = {
    0: "0xff0000",   # Red
    1: "0xffa500",   # Orange
    2: "0xffff00",   # Yellow
    3: "0x00ff00",   # Lime
    4: "0x00ffff",   # Cyan
    5: "0x0000ff",   # Blue
    6: "0xff00ff",   # Magenta
    7: "0xffc0cb",   # Pink
    8: "0x98ff98",   # Light Green
    9: "0xe6e6fa",   # Lavender
    10: "0xfdd9b5",  # Apricot
    11: "0xffe4b5",  # Moccasin
    12: "0x40e0d0",  # Turquoise
    13: "0xff6347",  # Tomato
    14: "0xff7f50",  # Coral
    15: "0xf0e68c",  # Khaki
    16: "0xff1493",  # Deep Pink
    17: "0xffb6c1",  # Light Pink
    18: "0x9370db",  # Medium Purple
    19: "0x00fa9a",  # Medium Spring Green
    20: "0x7fffd4"   # Aquamarine
}



# save a nodes df with x, y, z positions
pos = nx.fruchterman_reingold_layout(G_sample, dim=3)

# save to csv
df_nodes_pos = pd.DataFrame(pos).T

df_nodes_pos.columns = ["x", "y", "z"]
df_nodes_pos.index.name = "node_id"
df_nodes_pos.reset_index(inplace=True)
df_nodes_pos.head(2)

# map the cluster to the nodes as column
cluster_map = nx.get_node_attributes(G_sample, "cluster")
# same with title , year, doi
title_map = nx.get_node_attributes(G_sample, "title")
year_map = nx.get_node_attributes(G_sample, "year")
doi_map = nx.get_node_attributes(G_sample, "doi")

df_nodes_pos["cluster"] = df_nodes_pos["node_id"].map(cluster_map).astype(int)
df_nodes_pos["title"] = df_nodes_pos["node_id"].map(title_map)
df_nodes_pos["year"] = df_nodes_pos["node_id"].map(year_map).astype(int)
df_nodes_pos["doi"] = df_nodes_pos["node_id"].map(doi_map)
df_nodes_pos["color"] = df_nodes_pos["cluster"].map(color_dict)

df_nodes_pos.to_csv("/Users/jlq293/Downloads/05-animations/src/nodes.csv")
print(df_nodes_pos.head())

Number of nodes in the selected clusters: 2863
  node_id         x         y         z  cluster  \
0  n22951 -0.252518  0.684466 -0.498048       18   
1  n18117 -0.442171 -0.474197 -0.188537       16   
2  n24722  0.323195 -0.068103  0.063735       15   
3  n11595 -0.231222  0.570417 -0.416102       18   
4  n11879 -0.357088  0.191390  0.730870       17   

                                               title  year  \
0  Role of brain serotonin dysfunction in the pat...  2012   
1  5-HTTLPR polymorphism of serotonin transporter...  2008   
2  Saliva levels of 3-methoxy-4-hydroxyphenylglyc...  2013   
3  Depression in older survivors of myocardial in...  2003   
4  Selective Serotonin Reuptake Inhibitior-induce...  2003   

                                  doi     color  
0         10.1016/j.yjmcc.2012.08.006  0x9370db  
1          10.1177/030089160809400419  0xff1493  
2                    10.1002/hup.2273  0xf0e68c  
3                                 NaN  0x9370db  
4  10.1097/01.SMJ

In [22]:
G_sample.nodes(data=True)

NodeDataView({'n10026': {'doi': '10.1097/00127893-200211000-00006', 'year': 2002.0, 'title': 'Drug-induced parkinsonism', 'eid': '2-s2.0-0036854499', 'id': 'VanGerpen_2002', 'cluster': 48.0, 'centrality_alpha0.3_k20_res0.005': 0.0894860977941246}, 'n30674': {'doi': '10.1016/j.euroneuro.2016.11.014', 'year': 2017.0, 'title': 'Chronic vortioxetine treatment in rodents modulates gene expression of neurodevelopmental and plasticity markers', 'eid': '2-s2.0-85009784259', 'id': 'Waller_2017', 'cluster': 27.0, 'centrality_alpha0.3_k20_res0.005': 0.331784148113279}, 'n20000': {'doi': '10.1186/1744-859X-9-28', 'year': 2010.0, 'title': 'Sigma-1 receptor agonist fluvoxamine for postoperative delirium in older adults: Report of three cases', 'eid': '2-s2.0-77954587773', 'id': 'Furuse_2010_5', 'cluster': 132.0, 'centrality_alpha0.3_k20_res0.005': 0.363231076945872}, 'n6098': {'year': 1998.0, 'title': 'Sexual dysfunction and selective serotonin reuptake inhibitors [1]', 'eid': '2-s2.0-0031684179', '

In [20]:
# edge list to df
edges = nx.to_pandas_edgelist(G_sample)

#edges.to_csv("/Users/jlq293/Projects/Random Projects/LW-ThreeJS/ssrifromscratch/data/edges.csv", index=False)

print(edges.head())

   source  target    weight
0  n10026   n2082  0.650977
1  n10026   n5979  0.300000
2   n6098   n8167  0.664743
3   n6098  n19620  0.665278
4  n15785  n28173  0.300000


ValueError: Length mismatch: Expected axis has 8 elements, new values have 3 elements

In [15]:
df_nodes_pos.head(2)

,node_id,x,y,z,cluster,title,year,doi
0,n10026,0.047170,0.019802,0.675102,48.0,Drug-induced parkinsonism,2002.0,10.1097/00127893-200211000-00006
1,n30674,0.205122,-0.571663,0.828705,27.0,Chronic vortioxetine treatment in rodents modu...,2017.0,10.1016/j.euroneuro.2016.11.014


In [ ]:
import pandas as pd
import networkx as nx
G = nx.read_graphml("/Users/jlq293/Projects/Study-1-Bibliometrics/data/07-clustered-graphs/alpha0.3_k20_res0.005.graphml")

# keep a random sample of 1000 nodes
import random

# Get the list of nodes from the graph
all_nodes = list(G.nodes())

# Set a random seed for reproducibility
random.seed(42)

# Sample 1000 nodes randomly
sample_nodes = random.sample(all_nodes, 1000)

# Create a subgraph containing only the sampled nodes
G_sample = G.subgraph(sample_nodes)

# save the sample graph
nx.write_graphml(G_sample, "/Users/jlq293/Projects/Random Projects/LW-ThreeJS/SSRI/dat/directed_citation_graph_sample_1000.graphml")

# save a nodes df with x, y, z positions
pos = nx.fruchterman_reingold_layout(G_sample, dim=3)

# save to csv
df_nodes_pos = pd.DataFrame(pos).T
df_nodes_pos.columns = ["x", "y", "z"]
df_nodes_pos.reset_index(inplace=True)
df_nodes_pos.rename(columns={"index": "node_id"}, inplace=True)
df_nodes_pos.to_csv("/Users/jlq293/Projects/Random Projects/LW-ThreeJS/ssrifromscratch/data/nodes.csv")
print(df_nodes_pos.head())

# edge list to df
edges = nx.to_pandas_edgelist(G_sample)

# add columns for source and target positions
edges["source_x"] = edges["source"].map(df_nodes_pos["x"])
edges["source_y"] = edges["source"].map(df_nodes_pos["y"])
edges["source_z"] = edges["source"].map(df_nodes_pos["z"])
edges["target_x"] = edges["target"].map(df_nodes_pos["x"])
edges["target_y"] = edges["target"].map(df_nodes_pos["y"])
edges["target_z"] = edges["target"].map(df_nodes_pos["z"])

edges.to_csv("/Users/jlq293/Projects/Random Projects/LW-ThreeJS/ssrifromscratch/data/edges.csv", index=False)

print(edges.head())

In [1]:
import pandas as pd
import networkx as nx
G = nx.read_graphml("/Users/jlq293/Projects/Study-1-Bibliometrics/data/05-graphs/citation-graph/directed_citation_graph.graphml")

# keep a random sample of 1000 nodes
import random

# Get the list of nodes from the graph
all_nodes = list(G.nodes())

# Set a random seed for reproducibility
random.seed(42)

# Sample 1000 nodes randomly
sample_nodes = random.sample(all_nodes, 1000)

# Create a subgraph containing only the sampled nodes
G_sample = G.subgraph(sample_nodes)

# save the sample graph
nx.write_graphml(G_sample, "/Users/jlq293/Projects/Random Projects/LW-ThreeJS/SSRI/dat/directed_citation_graph_sample_1000.graphml")

# save a nodes df with x, y, z positions
pos = nx.fruchterman_reingold_layout(G_sample, dim=3)

# save to csv
df_nodes_pos = pd.DataFrame(pos).T
df_nodes_pos.columns = ["x", "y", "z"]
df_nodes_pos.reset_index(inplace=True)
df_nodes_pos.rename(columns={"index": "node_id"}, inplace=True)
df_nodes_pos.to_csv("/Users/jlq293/Projects/Random Projects/LW-ThreeJS/ssrifromscratch/data/nodes.csv")
print(df_nodes_pos.head())

# edge list to df
edges = nx.to_pandas_edgelist(G_sample)

# add columns for source and target positions
edges["source_x"] = edges["source"].map(df_nodes_pos["x"])
edges["source_y"] = edges["source"].map(df_nodes_pos["y"])
edges["source_z"] = edges["source"].map(df_nodes_pos["z"])
edges["target_x"] = edges["target"].map(df_nodes_pos["x"])
edges["target_y"] = edges["target"].map(df_nodes_pos["y"])
edges["target_z"] = edges["target"].map(df_nodes_pos["z"])

edges.to_csv("/Users/jlq293/Projects/Random Projects/LW-ThreeJS/ssrifromscratch/data/edges.csv", index=False)

print(edges.head())

           node_id         x         y         z
0       Banos_2010 -0.571330  0.024286 -0.871659
1  Rademacher_2002  0.778398 -0.454659  0.344531
2  DeBerardis_2014 -0.904431  0.219940 -0.367429
3    Gorelick_1992 -0.036906 -0.832864 -0.444589
4     Chen_2017_11 -0.128902 -0.691637 -0.071614
            source           target  source_x  source_y  source_z  target_x  \
0     Chen_2017_11   Kirchherr_2006       NaN       NaN       NaN       NaN   
1  Papakostas_2010     Blier_2001_2       NaN       NaN       NaN       NaN   
2     Knorr_2012_3   Loubinoux_2005       NaN       NaN       NaN       NaN   
3  AmadoLerma_2023  Sadegzadeh_2020       NaN       NaN       NaN       NaN   
4       Kelly_1996    Jenike_1990_2       NaN       NaN       NaN       NaN   

   target_y  target_z  
0       NaN       NaN  
1       NaN       NaN  
2       NaN       NaN  
3       NaN       NaN  
4       NaN       NaN  


In [4]:
df_nodes_pos

# make index the node_id
df_nodes_pos.reset_index(inplace=True)
df_nodes_pos.rename(columns={"index": "node_id"}, inplace=True)
df_nodes_pos

,node_id,x,y,z
0,Almohaimeed_2021_2,0.242494,-0.363186,0.816420
1,Moretti_2002,-0.072398,0.541191,-0.842076
2,Zehgeer_2018,0.009531,0.985944,-0.141192
3,Drubach_2015,0.572973,-0.695517,-0.257309
4,NoAuth_2001,-0.422828,-0.609002,0.690732
...,...,...,...,...
995,Dunner_1994,-0.841677,-0.521488,-0.256076
996,Bergen_2009,-0.241599,0.931078,0.205762
997,Bonne_2011,-0.826128,-0.032348,-0.498186
998,DaRocha_2007_2,-0.290279,0.072830,-0.904583
